In [ ]:
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
from tqdm.auto import tqdm
from dotenv import load_dotenv
import shutil

from utils import *

load_dotenv()

In [ ]:
filenames = index_directory(IMAGE_FOLDER)

In [ ]:
def tree_copy(
    source, source_prefix="/data/photos", destination_prefix="/data/local_photos"
):
    destination = source.replace(source_prefix, destination_prefix)
    if os.path.isfile(destination):
        pass
    else:
        try:
            shutil.copy(source, destination)

        except FileNotFoundError:
            destination_folder, _ = os.path.split(destination)
            os.makedirs(destination_folder)
            shutil.copy(source, destination)

        except OSError:
            pass

In [ ]:
pbar = tqdm(
    total=len(filenames),
    desc="Copying photos from remote to local",
)

futures = []
results = []

with ThreadPoolExecutor() as executor:
    for chunk in chunked(filenames, THREAD_CHUNKS):
        futures = []
        for filename in chunk:
            pbar.update()
            futures.append(executor.submit(tree_copy, filename))
        for future in as_completed(futures):
            try:
                infer_result = future.result()
                results.append(infer_result)
            except Exception as e:
                print("There was an exception: {}.".format(e))